# Quantum state preparation of elementary symmetric polynomial (ESP) state

An elementary symmetric polynomial ([ESP](https://en.wikipedia.org/wiki/Elementary_symmetric_polynomial)) function of an arbitrary vector $\mathbf{X}$ of dimension $M$ is defined as

$$ S_N^M (\mathbf{X})
= \sum_{1 \leq p_1 < \cdots < p_N \leq M} X_{p_1} \cdots X_{p_N},
$$
where $N \: (\leq M)$ is the degree of the polynomial and we define $S_0^M = 1$.
For example, $ S_M^M = X_1 \cdots X_M $ and
$ S_1^M = X_1 + \cdots + X_M $.
Any ESP can be recursively decomposed as

$$ S_N^M (\mathbf{X})
= S_{N - 1}^{M - 1} (\mathbf{X}_{- p})
+ S_N^{M - 1} (\mathbf{X}_{- p}), \quad
\forall \: p,
$$

where $\mathbf{X}_{- p}$ represents removing the $p$-th element from the vector.
This is why any ESP can be computed with $\mathcal{O} (M N)$ complexity.

An [ESP state](https://arxiv.org/abs/2301.09586) $|S_N^M\rangle$ is a state involving all $M$-qubit basis states that have $N$ Hamming weight and a specific coefficient structure

$$ |S_N^M\rangle
= \frac{1}{\sqrt{S_N^M}} \sum_{1 \leq p_1 < \cdots < p_N \leq M} \eta_{p_1} \cdots \eta_{p_N} \: | \cdots 0 \cdots 1_{p_1} \cdots 0 \cdots 1_{p_N} \cdots 0 \cdots \rangle,
$$
where the overlap $S_N^M$ is an ESP

$$ S_N^M (\mathbf{X})
= \sum_{1 \leq p_1 < \cdots < p_N \leq M} | \eta_{p_1} |^2 \cdots | \eta_{p_N} |^2,
$$
with all $ X_p = | \eta_p |^2 $.
As a specific example, the $|S_2^4\rangle$ state is

$$ |S_2^4\rangle
= \frac{1}{\sqrt{S_2^4}} \: \Big( \eta_1 \eta_2 |1100\rangle
+ \eta_1 \eta_3 |1010\rangle
+ \eta_1 \eta_4 |1001\rangle
+ \eta_2 \eta_3 |0110\rangle
+ \eta_2 \eta_4 |0101\rangle
+ \eta_3 \eta_4 |0011\rangle \Big),
$$
where the qubit indexing starts from $1$ and the left.
The overlap ESP is

$$ |S_2^4\rangle
= |\eta_1|^2 |\eta_2|^2
+ |\eta_1|^2 \eta_3|^2
+ |\eta_1|^2 \eta_4|^2
+ |\eta_2|^2 \eta_3|^2  
+ |\eta_2|^2 \eta_4|^2  
+ |\eta_3|^2 \eta_4|^2.
$$

In case of all $\eta_p = 1$, the ESP state reduces to the [Dicke state](https://arxiv.org/abs/1904.07358)
$$ |D_2^4\rangle
= \frac{1}{\sqrt{4 \choose 2}} \: \Big( |1100\rangle
+ |1010\rangle
+ |1001\rangle
+ |0110\rangle
+ |0101\rangle
+ |0011\rangle \Big).
$$

The state preparation algorithm is based on the fact that the ESP state can be recursively partitioned at its last qubit

$$ |S_j^p\rangle
= \Big( S_{j - 1}^{p - 1} / S_j^p \Big)^{1/2} \: \eta_p \:
| S_{j - 1}^{p - 1} \rangle \otimes |1\rangle
+ \Big( S_j^{p - 1} / S_j^p \Big)^{1/2} \:
| S_j^{p - 1} \rangle \otimes |0\rangle,
$$

where $p \leq M$ and $j \leq N$, with some additional subtleties due to the signs and complex phases of the ESP coefficients $\{ \eta_p \}$.

## Prerequisite

Let us install things.

In [1]:
!pip install qiskit==0.42.0
!pip install qiskit-aer==0.11.1

  Using cached qiskit_aer-0.11.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 12.8 MB/s eta 0:00:00
  Attempting uninstall: qiskit-aer
    Found existing installation: qiskit-aer 0.12.0
    Uninstalling qiskit-aer-0.12.0:
      Successfully uninstalled qiskit-aer-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
qiskit 0.42.0 requires qiskit-aer==0.12.0, but you have qiskit-aer 0.11.1 which is incompatible.


In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from qiskit import QuantumCircuit, QuantumRegister, Aer, transpile

## ESP

In [4]:
def espval(V, n):
    """
    Compute ESP using the recursion

    E[m, n] (V) = E[m-1, n] (V[1:m-1]) + V(m) E[m-1, n-1] (V[1:m-1]),

    where we start the indexing from one.

    Arguments:
    V -- vector of dimension m
    n -- ESP degree
    """
    # Check
    m = len(V)
    if m < 0 or n < 0:
        raise ValueError("m or n is less than 0 in espval")

    # Trivial cases
    if m < n:
        return 0
    if n == 0:
        return 1

    # Initialize
    E = np.zeros((m, n+1), dtype=type(V[0]))
    E[:, 0] = 1
    E[0, 1] = V[0]

    # SumESP
    for i in range(1, m):
        for j in range(1, n+1):
            E[i, j] = E[i-1, j] + ( V[i] * E[i-1, j-1] )

    return E[m-1, n]

In [5]:
def espangle(eta, q):
    """
    Compute the matrix element

    theta[p, q] = | eta[p] | sqrt( ESP(X[1:p-1], q-1) / ESP(X[1:p], q) ),

    where p is the dimension of eta vector, X[j] = | eta[j] |^2 for all j,
    and we start the indexing from one.

    Arguments:
    eta -- ESP vector of dimension p
    q -- degree index
    """
    # Check
    p = len(eta)
    if p < q or p < 2 or q < 1:
        raise ValueError("Invalid input dimensions in espangle")

    # Compute ESPs
    X = np.abs(eta)**2
    f1 = espval(X[0:p-1], q - 1)
    f2 = espval(X, q)

    # Factor
    if np.abs(eta[p-1]) < 1e-08:
        return 0.0
    f0 = np.sqrt(f1 / f2)

    return np.abs(eta[p-1]) * f0

## ESP state

The ESP state can be written as

$$ |S_N^M\rangle
= J_M \: U_{M, N} \: |0\rangle^{\otimes M - N} \otimes |1\rangle^{\otimes N},
$$
where we are indexing qubits starting from $1$ and the left.
Then the operator $U_{M, N}$ can be reconstructed from the split and cyclic shift (SCS) operators

$$ U_{M, N}
= \Big( \prod_{p = 2}^N \: SCS_{p, p - 1} \otimes \mathbb{I}^{\otimes M - p} \Big) \:
\Big( \prod_{p = N + 1}^M \: \mathbb{I}^{\otimes p - N - 1} \otimes SCS_{p, N} \otimes \mathbb{I}^{\otimes M - p} \Big),
$$
where $SCS_{p, q} \: (q < p)$ acts on $q + 1$ qubits with the last qubit index being $p$.
The $SCS_{pq}$ operator only encounters the following cases

\begin{align}
SCS_{pq} \: |0\rangle^{\otimes q + 1}
&= |0\rangle^{\otimes q + 1},
\\
SCS_{pq} \: |1\rangle^{\otimes q + 1}
&= |1\rangle^{\otimes q + 1},
\\
SCS_{pq} \: |0\rangle^{\otimes q - r + 1} \otimes |1\rangle^{\otimes r}
&= \theta_{pq} \: |0\rangle^{\otimes q - r + 1} \otimes |1\rangle^{\otimes r}
+ \sqrt{ 1 - \theta_{pq}^2 } \: |0\rangle^{\otimes q - r} \otimes |1\rangle^{\otimes r} \otimes |0\rangle,
\end{align}

where the $\{ \theta_{p, q} \}$ are scalars.
Let us now represent all the ESP state coefficients as,
$ \eta_p = | \eta_p | \: e^{i \alpha_p} $.
Then the $SCS$ coefficients are

\begin{align}
\theta_{pq}
&\equiv | \eta_{p} | \: \Big( S_{q - 1}^{p - 1} (\mathbf{X}_{-p}) / S_q^p (\mathbf{X}) \Big)^{1/2}, \quad
\\
\bar{\theta}_{pq}
&= \sqrt{ 1 - \theta_{pq}^2 }
= \Big( S_q^{p - 1} (\mathbf{X}_{- p}) / S_q^p (\mathbf{X}) \Big)^{1/2},
\end{align}
where all $ X_p = | \eta_p |^2 $ and $\mathbf{X}_{- p}$ represents removing the $p$-th element from the vector.
The $SCS$ circuit can be reconstructed from one two-qubit and $q - 1$ three-qubit gates

$$ SCS_{p, q}
= \Big( \prod_{r = 2}^q \: W_{p, r} \Big) \: T_p,
$$
where $T_p$ acts on qubit indices $p$ and $p - 1$, and $W_{p, q}$ acts on qubit indices $ p, p - q + 1 $, and $p - q$.
The $J_M$ operator will now take care of the ESP coefficient signs and complex phases

$$ J_M
= \bigotimes_{p = 1}^M R_z (\alpha_p).
$$

In [6]:
def ccry(circuit, angle, qc1, qc2, qt):
    circuit.cx(qc1, qt)
    circuit.ry(-angle / 4, qt)
    circuit.cx(qc2, qt)
    circuit.ry(angle / 4, qt)
    circuit.cx(qc1, qt)
    circuit.ry(-angle / 4, qt)
    circuit.cx(qc2, qt)
    circuit.ry(angle / 4, qt)

    return circuit

In [7]:
def scstwo(circuit, theta, p):
    circuit.cx(p - 2, p - 1)
    angle = 2 * np.arccos(theta)
    circuit.cry(angle, p - 1, p - 2)
    circuit.cx(p - 2, p - 1)

    return circuit

In [8]:
def scsthree(circuit, theta, p, q):
    circuit.cx(p - q - 1, p - 1)
    angle = 2 * np.arccos(theta)
    ccry(circuit, angle, p - q, p - 1, p - q - 1)
    circuit.cx(p - q - 1, p - 1)

    return circuit

In [9]:
def scs(circuit, eta, p, q):

    if p < q or p < 2 or q < 1:
        raise ValueError("Invalid input dimensions in scs")

    # Two-qubit gate
    theta_p1 = espangle(eta[0:p], 1)
    scstwo(circuit, theta_p1, p)

    # Three-qubit gates
    for r in range(2, q+1):
        theta_pr = espangle(eta[0:p], r)
        scsthree(circuit, theta_pr, p, r)

    return circuit

In [10]:
def espstate(eta, N):
    """
    Prepare ESP state circuit.

    Arguments:
    eta -- ESP vector of dimension M
    N -- Hamming weight
    """
    #--------------------------------------
    # Initialize
    #--------------------------------------

    M = len(eta)
    if M < N or M < 2 or N < 1:
        raise ValueError("Invalid input dimensions in espstate")

    # Circuit
    qr = QuantumRegister(M)
    qc = QuantumCircuit(qr)

    # Qubits
    for i in range(M - N, M):
        qc.x(i)

    #--------------------------------------
    # U(M, N)
    #--------------------------------------

    for p in range(M, N, -1):
        scs(qc, eta, p, N)

    for p in range(N, 1, -1):
        scs(qc, eta, p, p-1)

    #--------------------------------------
    # J(M)
    #--------------------------------------

    for p in range(M):
        qc.rz(np.angle(eta[p]), p)

    return qc

## Experiments

In [18]:
M = 4
N = 2

real_vec = np.random.rand(M)
imag_vec = np.random.rand(M)
etatest = real_vec + 1j * imag_vec
etatest

array([0.84060249+0.98429653j, 0.44298655+0.62214228j,
       0.84983142+0.91689094j, 0.59949146+0.63349094j])

In [20]:
qc = espstate(etatest, N)

In [21]:
qc.draw()

»
q7_0: ────────────────────────────────────────────────────────────»
                                        ┌───┐┌──────────────┐┌───┐»
q7_1: ───────────────────────────────■──┤ X ├┤ Ry(-0.44709) ├┤ X ├»
      ┌───┐     ┌────────────┐       │  └─┬─┘└──────────────┘└─┬─┘»
q7_2: ┤ X ├──■──┤ Ry(2.3023) ├──■────┼────■────────────────────┼──»
      ├───┤┌─┴─┐└─────┬──────┘┌─┴─┐┌─┴─┐                       │  »
q7_3: ┤ X ├┤ X ├──────■───────┤ X ├┤ X ├───────────────────────■──»
      └───┘└───┘              └───┘└───┘                          »
«                                                                   »
«q7_0: ─────────────────────────────────────────────────────────────»
«      ┌─────────────┐┌───┐┌──────────────┐┌───┐┌─────────────┐     »
«q7_1: ┤ Ry(0.44709) ├┤ X ├┤ Ry(-0.44709) ├┤ X ├┤ Ry(0.44709) ├──■──»
«      └─────────────┘└─┬─┘└──────────────┘└─┬─┘└─────────────┘  │  »
«q7_2: ─────────────────■────────────────────┼───────────────────┼──»
«                                            │                 ┌─┴─┐»
«q7_3: ──────────────────────────────────────■─────────────────┤ X ├»
«                                                              └───┘»
«                                             ┌───┐┌──────────────┐┌───┐»
«q7_0: ────────────────────────────────────■──┤ X ├┤ Ry(-0.24217) ├┤ X ├»
«                     ┌────────────┐       │  └─┬─┘└──────────────┘└─┬─┘»
«q7_1: ───────■───────┤ Ry(1.7539) ├──■────┼────■────────────────────┼──»
«           ┌─┴─┐     └─────┬──────┘┌─┴─┐┌─┴─┐                       │  »
«q7_2: ─────┤ X ├───────────■───────┤ X ├┤ X ├───────────────────────■──»
«      ┌────┴───┴────┐              └───┘└───┘                          »
«q7_3: ┤ Rz(0.81297) ├──────────────────────────────────────────────────»
«      └─────────────┘                                                  »
«      ┌─────────────┐┌───┐┌──────────────┐┌───┐┌─────────────┐     »
«q7_0: ┤ Ry(0.24217) ├┤ X ├┤ Ry(-0.24217) ├┤ X ├┤ Ry(0.24217) ├──■──»
«      └─────────────┘└─┬─┘└──────────────┘└─┬─┘└─────────────┘  │  »
«q7_1: ─────────────────■────────────────────┼───────────────────┼──»
«                                            │                 ┌─┴─┐»
«q7_2: ──────────────────────────────────────■─────────────────┤ X ├»
«                                                              └───┘»
«q7_3: ─────────────────────────────────────────────────────────────»
«                                                                   »
«                     ┌────────────┐     ┌─────────────┐
«q7_0: ───────■───────┤ Ry(2.0755) ├──■──┤ Rz(0.86398) ├
«           ┌─┴─┐     └─────┬──────┘┌─┴─┐├─────────────┤
«q7_1: ─────┤ X ├───────────■───────┤ X ├┤ Rz(0.95204) ├
«      ┌────┴───┴────┐              └───┘└─────────────┘
«q7_2: ┤ Rz(0.82334) ├──────────────────────────────────
«      └─────────────┘                                  
«q7_3: ─────────────────────────────────────────────────
«

In [22]:
qc.depth()

31

In [23]:
# Aer backend
backend = Aer.get_backend('statevector_simulator')
circ = transpile(qc, backend)

# Result
result = backend.run(circ).result()

In [24]:
counts = result.get_counts()
counts

{'0011': 0.131805399779043,
 '0101': 0.353162438002868,
 '0110': 0.122951270774265,
 '1001': 0.171892331083804,
 '1010': 0.059843228692777,
 '1100': 0.160345331667244}

In [25]:
statevector = result.get_statevector()
statevector

Statevector([ 0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              3.61585451e-01+3.25785383e-02j,
              0.00000000e+00+0.00000000e+00j,
              5.93826392e-01-2.30792938e-02j,
              3.50219487e-01+1.72505577e-02j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              4.14096965e-01-2.03969335e-02j,
              2.44444208e-01+9.50041923e-03j,
              0.00000000e+00+0.00000000e+00j,
              3.98815939e-01-3.59329733e-02j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
             -4.29487178e-18+2.74212701e-17j],
            dims=(2, 2, 2, 2))
